# Aula 5: Solução dos exercícios

Este notebook apresenta a solução dos problemas propostos na aula 5.

A solução utiliza as subrotinas apresentadas na aula 5 e repetidas neste notebook.

-----

<!-- TEASER_END -->

In [ ]:
using PyPlot

In [ ]:
function linear_fit(x, y)

    
    sx = sum(x)
    sy = sum(y)

    m = length(x)

    sx2 = zero(sx)
    sy2 = zero(sy)
    sxy = zero(sx*sy)

    for i = 1:m
        sx2 += x[i]*x[i]
        sy2 += y[i]*y[i]
        sxy += x[i]*y[i]
    end

    a0 = (sx2*sy - sxy*sx) / ( m*sx2 - sx*sx )
    a1 = (m*sxy - sx*sy) / (m*sx2 - sx*sx)

    return (a0, a1)
end

In [ ]:
function least_squares(y, ϕ)
    m = length(y)
    n = size(ϕ, 2)
    
    if size(ϕ,1) != m
        error("O número de linhas de ϕ deve ter o mesmo comprimento de x e y")
    end
    A = zeros(n,n)
    b = zeros(n)
    s = 0.0
    for i = 1:n
        for j in 1:i
            s = 0.0
            for k = 1:m
                s += ϕ[k,i] * ϕ[k,j]
            end
            A[j,i] = s
            A[i,j] = s
        end
        s = 0.0
        for k = 1:m
            s += y[k] * ϕ[k,i]
        end
        b[i] = s
    end
    
    return A\b
    
end


In [ ]:
function poly_fit(x, y, ndeg)
    n = ndeg + 1
    m = length(x)
    
    ϕ = zeros(m, n)
    ϕ[:,1] .= 1.0
    for i = 2:n
        ϕ[:,i] .= x .^ (i-1)
    end
    
    return least_squares(y, ϕ)
end


# Pacotes

 * <https://github.com/pjabardo/CurveFit.jl> Algumas funções para ajuste de curva simples
 * <https://github.com/JuliaNLSolvers/LsqFit.jl> Um pacote mais profissa e completo
 * <https://github.com/JuliaStats/GLM.jl> Um pacote geral de estatística para modelos lineares
 ------

# Problemas




# Problema 1:

Ache uma interpolação polinomial para a curva de calibração de um anemômetro de fio quente onde para diferentes velocidades $U_i$ se conhece a resposta $E_i$ em volts do anemômetro.

In [ ]:
Uc= [0.495, 0.637, 0.809, 1.058, 1.331, 1.702, 2.173, 2.786, 3.561, 4.508, 5.780, 7.417, 9.449, 12.272, 
    15.637, 20.074];
Ec = [1.46, 1.485, 1.511, 1.543, 1.574, 1.609, 1.646, 1.688, 1.732, 1.778, 1.828, 1.885, 1.944, 
    2.014, 2.084, 2.16];


In [ ]:
plot(Uc, Ec, "bo")

In [ ]:
using Polynomials

In [ ]:
ee = range(1.3, 2.3, length=101)
p1 = Poly(poly_fit(Ec, Uc, 2))
plot(Uc, Ec, "bo")
plot(p1.(ee), ee, "r-")


In [ ]:
p1 = Poly(poly_fit(Ec, Uc, 3))
plot(Uc, Ec, "bo")
plot(p1.(ee), ee, "r-")


In [ ]:
p1 = Poly(poly_fit(Ec, Uc, 4))
plot(Uc, Ec, "bo")
plot(p1.(ee), ee, "r-")


In [ ]:
p1 = Poly(poly_fit(Ec, Uc, 5))
plot(Uc, Ec, "bo")
plot(p1.(ee), ee, "r-")


In [ ]:
p1 = Poly(poly_fit(Ec, Uc, 6))
plot(Uc, Ec, "bo")
plot(p1.(ee), ee, "r-")


In [ ]:
p1 = Poly(poly_fit(Ec, Uc, 7))
plot(Uc, Ec, "bo")
plot(p1.(ee), ee, "r-")



In [ ]:
p1 = Poly(poly_fit(Ec, Uc, 10))
plot(Uc, Ec, "bo")
plot(p1.(ee), ee, "r-")


# Problema 2

O perfil de velocidades em uma camada limite turbulenta rugosa pode ser representada por uma lei de potência:

$$
\frac{u(z)}{u_{ref}} = \left(\frac{z}{z_{ref}}\right)^\alpha
$$

A partir dos dados medidos em túnel de vento abaixo, calcule o expoente $\alpha$ do perfil de velocidades. A velocidade está em m/s e as alturas estão em mm.

In [ ]:
u = [6.36,  6.65 , 6.95,  6.97 , 7.13 , 7.30 , 7.62,  7.70 , 7.68,  7.86 , 7.97,  8.30,
     8.12,  8.75,  8.64,  9.01,  9.16 , 9.38,  9.60,  9.72,  9.77, 10.02, 10.30, 10.30,
    10.74, 11.04, 11.38, 11.68, 11.44, 11.67, 11.80, 12.06, 12.24, 12.52]
z = [ 80,  90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 220, 240, 260, 280, 300, 320,
     340, 360, 380, 400, 420, 460, 500, 540, 580, 600, 620, 660, 700, 740, 780];

In [ ]:
zref = 50.0
logu = log.(u)
logz = log.(z/zref)
fit = linear_fit(logz, logu)
α = fit[2]
uref = exp(fit[1])

zz = 1:1000
ufun(z, α, uref, zref=10) = uref * (z/zref)^α
uu = ufun.(zz, α, uref, zref)

plot(u,z, "bo")
plot(uu, zz, "r-")


println("α = ", round(α, digits=3))

# Problema 3

A região próxima ao solo de uma camada limite turbulenta pode ser representada pela lei da parede:

$$
\frac{u(z)}{u_*} = \frac{1}{\kappa}\ln\frac{z}{z_0}
$$

nesta equação $\kappa \approx 0.4$ é a constante de von Karman, $u_*$ é a velocidade de atrito que vale
$$
u_* = \sqrt{\frac{\tau_w}{\rho}}
$$
onde $\tau_w$ é a tensão de cisalhamento na parede e $\rho$ é a densidade do ar.
e $z_0$ é o comprimento de rugosidade.

A partir do perfil de velocidades do problema anterior, estime a velocidade de atrito $u_*$ e comprimento de rugosidade $z_0$. Use apenas os pontos abaixo de $z=400$ mm.

In [ ]:
κ = 0.4
idx = z .< 400
z1 = z[idx]
u1 = u[idx]
logz = log.(z1)

fit = linear_fit(logz, u1)
uₜ = fit[2] * κ
z₀ = exp(-fit[1]/fit[2])
ufun2(z, uₜ, z₀, κ=0.4) = uₜ/κ * log(z/z₀)

uu2 = ufun2.(zz, uₜ, z₀)

plot(u, z, "bo")
plot(u1, z1, "rs")
plot(uu2, zz, "r-")

println("Modelo z₀ = ", round(z₀, digits=2), " mm")
println("Protótipo z₀ = ", round(z₀/5, digits=2), " m")


In [ ]:
semilogy(u, z, "bo")
plot(u1, z1, "rs")
zz3 = 80:800
uu3 = ufun2.(zz3, uₜ, z₀)
plot(uu3, zz3, "r-")


# Problema 4

Um modelo semi-empírico para o comportamento de um anemômetro de fio quente é dado pela lei de King:

$$
E^2 = A + B \cdot \sqrt{U}
$$
onde E é a tensão no anemômetro de U a velocidade do ar.

Use a curva de calibração do problema 1 para estimar as constantes A e B.

Na prática o desempenho da lei de King não é muito boa. A lei de King modificada apresenta resultados melhores:
$$
E^2 = A + B \cdot U^n
$$
Usando a mesma curva de calibração, estime as consantes A, B e n. (cuidado, este é um problema não linear)


In [ ]:
E2 = Ec.*Ec
sU = sqrt.(Uc)
plot(sU, E2, "bo")
fit = linear_fit(E2, sU)
sUcalc = fit[1] .+ fit[2] .* E2
plot(sUcalc, E2, "r-")

In [ ]:
ucalc = sUcalc.^2
plot(Uc, Ec, "bo")
plot(ucalc, Ec, "r-")

### Método dos mínimos quadrados não linear

Até o momento, tratamos de ajuste de curva do seguinte formato:

$$
y \approx y^\delta = \sum_{i=1}^n a_i\phi_i(x)
$$
Este ajuste é linear nos coeficientes $a_i$. Mas o que fazer quando esta linearidade não existe. Em alguns casos pode-se transformar a equação e se obter uma outra equação com outras variáveis que é linear. Isto foi visto ao se ajustar uma lei de potência na camada limite ou a lei da parede (log). Também se usou este expediante para se ajustar a lei de King. 

No entanto, esta abordagem nem sempre é conveniente ou possível. Uma outra possibilidade é tentar fazer o ajuste não linear diretamente. Neste caso, vamos utilizar um processo iterativo inspirado no método de Newton-Raphson para a solução de problemas não lineares. A equação de ajuste neste caso tem a seguinte forma:

$$
y \approx y^\delta = f(x, a_1, \ldots, a_n)
$$

A partir de estimativas iniciais dos coeficientes $a_i$, $a_i^{(0)}$, expandindo este problema com uma série de Taylor, podemos chegamos à seguinte relação:

$$
y^\delta =  f(x, a_1, \ldots, a_n) \approx   f(x, a_1^{(0)}, \ldots, a_n^{(0)}) + \sum_{i=1}^n \Delta a_i\left.\frac{\partial f}{\partial a_i}\right|_{a_1^{(0)}, \ldots, a_n^{(0)}}
$$

Agora basta fazer o *ajuste linear* de 
$$
\tilde{y} = y -  f(x, a_1^{(0)}, \ldots, a_n^{(0)})  = \sum_{i=1}^n \Delta a_i\left.\frac{\partial f}{\partial a_i}\right|_{a_1^{(0)}, \ldots, a_n^{(0)}}
$$

Calculado $\Delta a_i$, 
$$
a_i^{(1)} = a_i^{(0)} + \Delta a_i
$$
O processo pode ser repetido até que $\Delta a_i$ seja muito pequeno (critério de convergência).

Observe que nas equações acima, a matrix $\phi$ usada para fazer o ajuste tem colunas da seguinte forma:

$$
\phi[:,i] = \left.\frac{\partial f}{\partial a_i}\right|_{a_1^{(0)}, \ldots, a_n^{(0)}}
$$

Em cada iteração usa-se a função para ajuste linear desenvolvida acima para se convergir a solução.

In [ ]:
## Mínimos quadrados não linear
function fit_king(E, U)
    E2 = E .* E
    sU = sqrt.(U)
    fit = linear_fit(sU, E2)
    A = fit[1]
    B = fit[2]
    
    return A,B
end

function fit_mod_king(E, U, err=1e-8, maxiter=100)
    np = length(E)
    m = 3
    
    A0, B0 = fit_king(E, U)
    n0 = 0.5
    
    E2 = E .* E
    
    ϕ = zeros(np, m)
    ϕ[:,1] .= 1.0
    
    
    y = zeros(np)
    niter = 1
    converged = false
    for i = 1:maxiter
        @. y = E2 - (A0 + B0 * U^n0)
        ϕ[:, 2] .= U.^n0
        ϕ[:, 3] .= U.^n0 .* B0 .* log.(U)
        fit = least_squares(y, ϕ)
        
        dA, dB, dn = fit[1], fit[2], fit[3]
        A0 = A0 + dA
        B0 = B0 + dB
        n0 = n0 + dn
        niter = i
        if max(abs(dA), abs(dB), abs(dn)) < err
            converged = true
            break
        end
        
    end
    
    return A0, B0, n0, niter, converged
    
end

In [ ]:
A, B = fit_king(Ec, Uc)

In [ ]:
A1, B1, n1, niter, converged = fit_mod_king(Ec, Uc, 1e-12)

In [ ]:
king(E, A, B) = ( (E^2-A)/B )^2
modking(E, A, B, n) = ( (E^2-A)/B)^(1/n)
p4 = Poly(poly_fit(Ec, Uc, 4))
p5 = Poly(poly_fit(Ec, Uc, 5))
p6 = Poly(poly_fit(Ec, Uc, 6))


Uk = king.(Ec, A, B)
Umk = modking.(Ec, A1, B1, n1);

Up4 = p4.(Ec)
Up5 = p5.(Ec)
Up6 = p6.(Ec)

errk = (Uk - Uc) ./ Uc
errmk = (Umk - Uc) ./ Uc

errp4 = (Up4 - Uc) ./ Uc
errp5 = (Up5 - Uc) ./ Uc
errp6 = (Up6 - Uc) ./ Uc

plot(Uc, Ec, "bo")
plot(Uk, Ec, "r-")
plot(Umk, Ec, "g--")

In [ ]:
plot(Uc, errk*100, "r-")
plot(Uc, errmk*100, "b--")
plot(Uc, errp4*100, "g-")
plot(Uc, errp5*100, "g--")



In [ ]:
plot(Uc, errmk*100, "b-")
plot(Uc, errp4*100, "g-")
plot(Uc, errp5*100, "g--")
plot(Uc, errp6*100, "g:")
axhline(0.0)
